In [0]:
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
import os, shutil
tensorflow.__version__

'2.2.0'

In [0]:
FAST_RUN = False
IMAGE_WIDTH=150
IMAGE_HEIGHT=150
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

nb_train_samples = 18700
nb_validation_samples = 4700
epochs = 50
batch_size = 16

## **Data preprocessing without using Google Drive**

In [0]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  34.9M      0  0:00:22  0:00:22 --:--:--  121M


In [0]:
!unzip -q kagglecatsanddogs_3367a.zip
!ls PetImages

Cat  Dog


In [0]:

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        fobj = open(fpath, "rb")
        if tf.compat.as_bytes("JFIF") not in fobj.peek(10):
            num_skipped += 1
            # Delete corrupted image
            os.system("rm " + fpath)
print("Deleted %d images" % num_skipped)


Deleted 1590 images


In [0]:

base_dir = '/content/PetImages'
dest = os.path.join(base_dir,'train')
os.mkdir(dest)

CATEGORIES = ['Dog','Cat']

for c in CATEGORIES:
  dest = os.path.join(base_dir,'train',c)
  os.mkdir(dest)

  for i in range(0,10000):
      try:
        src=os.path.join(base_dir,c)
        src=os.path.join(src,'{}.jpg'.format(i))
        shutil.move(src,dest)
      except Exception as e:
        pass


In [0]:

base_dir = '/content/PetImages'
dest = os.path.join(base_dir,'val')
os.mkdir(dest)

CATEGORIES = ['Dog','Cat']

for c in CATEGORIES:
  dest = os.path.join(base_dir,'val',c)
  os.mkdir(dest)

  for i in range(10000,12500):
      try:
        src=os.path.join(base_dir,c)
        src=os.path.join(src,'{}.jpg'.format(i))
        shutil.move(src,dest)
      except Exception as e:
        pass


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'val')

train_generator = train_datagen.flow_from_directory(train_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 18708 images belonging to 2 classes.
Found 4702 images belonging to 2 classes.


# **Model**

In [0]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [0]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

In [0]:
# epochs=3 if FAST_RUN else 50

# history = model.fit_generator(
#       train_generator,
#       steps_per_epoch=nb_train_samples//batch_size,
#       epochs=epochs,
#       validation_data=validation_generator,
#       validation_steps=nb_validation_samples//batch_size)

In [0]:
model.save_weights("model.h5")

In [0]:
model.evaluate_generator(validation_generator,steps=nb_validation_samples//batch_size)

[0.2862638533115387, 0.8264126777648926]

In [0]:
# import pickle
# from google.colab import files


# output = open('/trainHistoryDict', 'wb') 
# pickle.dump(history.history, output)
# output.close()

# #Upload on google drive
# # file = drive.CreateFile({'title': 'test'})
# # file.SetContentFile('/trainHistoryDict')
# # file.Upload()

# #downloand on local machine
# files.download('/trainHistoryDict')

# **Transfer learning**

In [0]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [0]:
conv_base.trainable=False

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
from tensorflow.keras import optimizers


model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-5),metrics=['acc'])

epochs=3 if FAST_RUN else 30

history = model.fit_generator(
      train_generator,
      steps_per_epoch=nb_train_samples//batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=nb_validation_samples//batch_size)

In [0]:
model.save_weights('model4.h5')

In [0]:
model.evaluate_generator(validation_generator,steps=nb_validation_samples//batch_size)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.252545565366745, 0.9080631136894226]

In [0]:
from google.colab import files

files.download('model4.h5')

In [0]:
model.load_weights('model2.h5')